# Import Librairies et données

In [1]:
pip install .

Processing c:\users\15girav\moi\projet 5 oc
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/

  error: subprocess-exited-with-error
  
  Preparing metadata (pyproject.toml) did not run successfully.
  exit code: 1
  
  [21 lines of output]
  + C:\Users\15GIRAV\AppData\Local\anaconda3\python.exe C:\Users\15GIRAV\AppData\Local\Temp\pip-install-6ls15a4l\numpy_8fba3be4b59a495e8721692abe313581\vendored-meson\meson\meson.py setup C:\Users\15GIRAV\AppData\Local\Temp\pip-install-6ls15a4l\numpy_8fba3be4b59a495e8721692abe313581 C:\Users\15GIRAV\AppData\Local\Temp\pip-install-6ls15a4l\numpy_8fba3be4b59a495e8721692abe313581\.mesonpy-lkeuxqpn -Dbuildtype=release -Db_ndebug=if-release -Db_vscrt=md --native-file=C:\Users\15GIRAV\AppData\Local\Temp\pip-install-6ls15a4l\numpy_8fba3be4b59a495e8721692abe313581\.mesonpy-lkeuxqpn\meson-python-native-file.ini
  The Meson build system
  Version: 1.2.99
  Source dir: C:\Users\15GIRAV\AppData\Local\Temp\pip-install-6ls15a4l\numpy_8fba3be4b59a495e8721692abe313581
  Build dir: C:\Users\15GIRAV\AppData\Local\Temp\pip-install-6ls15a4l\numpy_8fba3be4b59a495

In [2]:
import joblib

In [4]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

In [18]:
sirh=pd.read_csv('extrait_sirh.csv')

In [17]:
eval=pd.read_csv('extrait_eval.csv')

In [19]:
sondage=pd.read_csv('extrait_sondage.csv')

# Preprocessing

In [29]:
import pandas as pd

def prepare_data_for_prediction(df_sirh, df_eval, df_sondage):
    """
    Réalise la jointure et le nettoyage des 3 sources de données TechNova.
    """
    # 1. Nettoyage des IDs (gestion des types et des zéros non significatifs)
    df_sondage['code_sondage'] = df_sondage['code_sondage'].astype(int)
    
    # 2. Jointures successives
    # Jointure SIRH + Evaluation (sur id_employee / eval_number)
    df_merged = pd.merge(df_sirh, df_eval, left_on='id_employee', 
                         right_on=df_eval['eval_number'].str.replace('E_', '').astype(int), 
                         how='inner')
    
    # Jointure avec le Sondage
    data = pd.merge(df_merged, df_sondage, left_on='id_employee', 
                        right_on='code_sondage', how='inner')

    # 3. Nettoyage de la colonne salaire (suppression du % et conversion en float)
    if 'augementation_salaire_precedente' in data.columns:
        data['augmentation_salaire_precedente_pourcentage'] = (
            data['augementation_salaire_precedente']
            .str.replace(' %', '')
            .astype(float) / 100
        )

    # 4. Suppression des colonnes de jointure inutiles pour le modèle
    cols_to_drop = ['eval_number', 'code_sondage', 'augementation_salaire_precedente', 'key_0','ayant_enfants']
    data = data.drop(columns=[c for c in cols_to_drop if c in data.columns])

    return data

In [30]:
prepare_data_for_prediction(sirh, eval, sondage)

,id_employee,age,genre,revenu_mensuel,statut_marital,departement,poste,nombre_experiences_precedentes,nombre_heures_travailless,annee_experience_totale,...,nombre_participation_pee,nb_formations_suivies,nombre_employee_sous_responsabilite,distance_domicile_travail,niveau_education,domaine_etude,frequence_deplacement,annees_depuis_la_derniere_promotion,annes_sous_responsable_actuel,augmentation_salaire_precedente_pourcentage
0,1,41,F,5993,Célibataire,Commercial,Cadre Commercial,8,80,8,...,0,0,1,1,2,Infra & Cloud,Occasionnel,0,5,0.11
1,2,49,M,5130,Marié(e),Consulting,Assistant de Direction,1,80,10,...,1,3,1,8,1,Infra & Cloud,Frequent,1,7,0.23
2,4,37,M,2090,Célibataire,Consulting,Consultant,6,80,7,...,0,3,1,2,2,Autre,Occasionnel,0,0,0.15
3,5,33,F,2909,Marié(e),Consulting,Assistant de Direction,1,80,8,...,0,3,1,3,4,Infra & Cloud,Frequent,3,0,0.11
4,7,27,M,3468,Marié(e),Consulting,Consultant,9,80,6,...,1,3,1,2,1,Transformation Digitale,Occasionnel,2,2,0.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,2061,36,M,2571,Marié(e),Consulting,Consultant,4,80,17,...,1,3,1,23,2,Transformation Digitale,Frequent,0,3,0.17
1466,2062,39,M,9991,Marié(e),Consulting,Manager,4,80,9,...,1,5,1,6,1,Transformation Digitale,Occasionnel,1,7,0.15
1467,2064,27,M,6142,Marié(e),Consulting,Tech Lead,1,80,6,...,1,0,1,4,3,Infra & Cloud,Occasionnel,0,3,0.20
1468,2065,49,M,5390,Marié(e),Commercial,Cadre Commercial,2,80,17,...,0,3,1,2,3,Transformation Digitale,Frequent,0,8,0.14


# Encodage et modélisation

In [32]:
# 1. Identification des colonnes
# Liste des variables numériques nécessitant un scaling
numeric_features = [
    'age', 'revenu_mensuel', 'annee_experience_totale', 
    'annees_dans_l_entreprise', 'distance_domicile_travail',
    'augmentation_salaire_precedente_pourcentage'
]

# Liste des variables catégorielles à encoder
categorical_features = [
    'statut_marital', 'departement', 'poste', 
    'domaine_etude', 'frequence_deplacement', 'heure_supplementaires'
]

# 2. Définition des transformateurs
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore')) #si une donnée inconnue apparait (nouveau poste par exemple), elle sera prise en compte)
])

# 3. Assemblage du preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# 4. Création du pipeline complet avec le modèle final (plus besoin de réécrire le code de nettoyage)
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(
        n_estimators=100, 
        random_state=42,
        class_weight='balanced'
    ))
])

In [33]:
X=data.drop(columns=['a_quitte_l_entreprise', 'id_employee'])
y=data['a_quitte_l_entreprise']

In [34]:
# 5. Entraînement sur TOUTE la donnée (X et y doivent être préparés en amont)
model_pipeline.fit(X, y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['age', 'revenu_mensuel',
                                                   'annee_experience_totale',
                                                   'annees_dans_l_entreprise',
                                                   'distance_domicile_travail',
                                                   'augmentation_salaire_precedente_pourcentage']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['statut_marital',
                                                   'departement', 'poste',
                                                   'domaine_etude',
                                                   'frequence_deplacement',
                                                   'heure_supplementaires'])])),
                ('classifier',
                 RandomForestClassifier(class_weight='balanced',
                                        random_state=42))])

# Sauvegarde du Pipeline et Test

In [39]:
import joblib

# Créer un dossier 'models' s'il n'existe pas
import os
if not os.path.exists('models'):
    os.makedirs('models')

# Sauvegarder le pipeline complet (Preprocessing + Modèle)
joblib.dump(model_pipeline, 'final_pipeline_turnover.pkl')

['final_pipeline_turnover.pkl']

In [38]:
# Simulation : on charge le modèle comme le ferait l'application
loaded_model = joblib.load('models/final_pipeline_turnover.pkl')

# On prend une ligne au hasard dans X pour tester
sample_data = X.iloc[[0]] 

# Prédiction
prediction = loaded_model.predict(sample_data)
proba = loaded_model.predict_proba(sample_data)

print(f"Prédiction : {'Départ' if prediction[0] == 1 else 'Reste'}")
print(f"Probabilité de départ : {proba[0][1]:.2%}")

Prédiction : Départ
Probabilité de départ : 72.00%


# FastAPI

In [40]:
pip install fastapi uvicorn joblib pandas


   ---------------- ----------------------- 2/5 [uvicorn]
   ------------------------ --------------- 3/5 [starlette]
   -------------------------------- ------- 4/5 [fastapi]
   -------------------------------- ------- 4/5 [fastapi]
   ---------------------------------------- 5/5 [fastapi]

Note: you may need to restart the kernel to use updated packages.
